In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.optim.lr_scheduler import CosineAnnealingLR


In [2]:
# Importing all packages
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from torchvision import models
from PIL import Image
import cv2
import os
import pandas as pd
from skimage import exposure
import PIL.Image as Image 
import skimage.feature as skf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# Set device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {'GPU' if device.type == 'cuda' else 'CPU'}")


Using device: GPU


In [4]:
from efficientnet_pytorch import EfficientNet

# Load the model architecture
model = EfficientNet.from_pretrained('efficientnet-b0')

# Replace the final layer to match your class count
num_features = model._fc.in_features
model._fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 5)  # Replace 5 with your number of classes
)

# Load the weights
model.load_state_dict(torch.load('best_efficientnet_dr_detection2.pth'))

# Set the model to evaluation mode
model = model.to(device)
model.eval()


Loaded pretrained weights for efficientnet-b0


/tmp/ipykernel_925/2971219377.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_efficientnet_dr_detection2.pth'))


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [5]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Path to the image
image_path = './Datasets/eye-q/test/37359_right.jpeg'

# Define the preprocessing transformations (same as validation transforms)
image_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the image
image = Image.open(image_path).convert("RGB")  # Convert to RGB in case it's not

# Apply transformations
input_tensor = image_transforms(image).unsqueeze(0)  # Add batch dimension

# Move the tensor to the device (GPU or CPU)
input_tensor = input_tensor.to(device)

# Generate predictions
with torch.no_grad():
    outputs = model(input_tensor)  # Forward pass
    _, predicted_class = torch.max(outputs, 1)  # Get the predicted class index

# Map the predicted class index to the class name
class_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']  # Replace with your actual class names
predicted_label = class_names[predicted_class.item()]

print(f"Predicted class for the image '{image_path}': {predicted_label}")


Predicted class for the image './Datasets/eye-q/test/37359_right.jpeg': Class 2
